In [1]:
using Revise, TaylorModels, Plots, TaylorSeries, IntervalArithmetic, IntervalOptimisation



In [2]:

function branchandbound(p::Taylor1{T},dom::Interval{T},ϵ::Number)where{T}
    Rperv = evaluate(p,dom)
    D = bisect(dom)
    D = [ D[i][1] for i = 1:2]
    R = [evaluate(p, D[i]) for i = 1:length(D)]
    Rnext = Interval(minimum(R[i].lo for i=1:length(R)),
                     maximum(R[i].hi for i=1:length(R)))
    K = 1  
    #condition given in the algo and max no of steps K
    while  (Rperv.hi - Rnext.hi) <= ϵ*(Rnext.hi - Rnext.lo) && 
           (Rperv.lo - Rnext.lo) <= ϵ*(Rnext.hi - Rnext.lo) && (K <= 100)
        #caclulate the max and min of range before any further splitting
        Rperv = Interval(minimum(R[i].lo for i=1:length(R)),  
                         maximum(R[i].hi for i=1:length(R)))
       
        max = maximum(R[i].hi for i=1:length(R)) # max of range 
        #index of max range giving domain 
        for i = 1:length(R)
            if R[i].hi == max
                max = i
                break;
            end
        end
        min = minimum(R[i].lo for i=1:length(R))# min of range
        #index of min range giving domain 
        for i = 1:length(R)
            if R[i].lo == min
                min = i
                break;
            end
        end
          
        K = K + 1   #update value of K 
        if min == max             
            D1, D2 = bisect(D[max]) #privide tuple bisecting domain
            D[max] = D1             #set the first half of selected domain to itself  
            DD = append!(D[1:max], D2)#append the second half of domain.
            D = append!(DD, D[(max+1):length(D)])#update the array of domain
            #do the same for the range 
            R[max] = evaluate(p,D[max])                  
            RR = append!(R[1:max], evaluate(p,D[max+1]))
            R = append!(RR, R[(max+1):length(R)])
            #max and min of range after split. 
            Rnext = Interval(minimum(R[i].lo for i=1:length(R)),
                             maximum(R[i].hi for i=1:length(R)))
        else
            #the steps are same as above 
            D1, D2 = bisect(D[max])
            D[max] = D1
            DD = append!(D[1:max], D2)
            D = append!(DD, D[(max+1):length(D)])
            R[max] = evaluate(p,D[max])
            RR = append!(R[1:max], evaluate(p,D[max+1]))
            R = append!(RR, R[(max+1):length(R)])
            # if index 'max' occurs before the index 'min' then
            #update 'min' as the indexing have changed after the update in the domain array. 
            if max < min
                min = min + 1
            end
            #the steps are same as above but according to the 'min' as index
            D1, D2 = bisect(D[min])
            D[min] = D1
            DD = append!(D[1:min], D2)
            if (min + 1) == length(D)
                D = append!(DD, D[min+1])
            else
                D = append!(DD, D[(min+1):length(D)])
            end
            R[min] = evaluate(p,D[min])
            RR = append!(R[1:min], evaluate(p,D[min+1]))
            R = append!(RR, R[(min+1):length(R)])
            Rnext = Interval(minimum(R[i].lo for i=1:length(R)),
                             maximum(R[i].hi for i=1:length(R)))
        end
    end
    return Interval(minimum(R[i].lo for i=1:length(R)), #return the interval range.
                    maximum(R[i].hi for i=1:length(R)))
end

branchandbound (generic function with 1 method)

In [3]:

ϵ=0.4
Dx=Interval(-4.5,-0.3)
dom=Dx
x=Taylor1(7)

sin=x - (x*x*x)/6.0 + (x*x*x*x*x)/120.0 - (x*x*x*x*x*x*x)/5040.0

println("range before branch and bound")
println(evaluate(sin,dom))
println("range using branch and bound")
println(branchandbound(sin,Dx,ϵ))

function _minmax(p, dom) #takes time
    global_min, _ = minimise(p, dom)
    minus_global_max, _ = minimise(-p, dom)
    global_max = -minus_global_max
    return global_min, global_max
end

println(_minmax(sin, Dx))



range before branch and bound
[-4.6569, 10.6522]
range using branch and bound
[-1.00207, 2.72424]
([-1.00057, -0.999843], [2.72256, 2.72754])


In [4]:
Du = Interval(-4.5,-0.3)
dom = Du
u = Taylor1(3)

bspline0 = (1 - u) * (1 - u) * (1 - u) / 6.0

 0.16666666666666666 - 0.5 t + 0.5 t² - 0.16666666666666666 t³ + 𝒪(t⁴)

In [6]:
println(branchandbound(bspline0,dom,ϵ))
println(_minmax(bspline0, dom))

[0.366166, 27.7292]
([0.366166, 0.366407], [27.7221, 27.7292])


In [7]:
bspline1 = (3 * u*u*u - 6 * u*u + 4) / 6.0

 0.6666666666666666 - 1.0 t² + 0.5 t³ + 𝒪(t⁴)

In [8]:
println(branchandbound(bspline1,dom,ϵ))
println(_minmax(bspline1, dom))

[-65.1459, 0.563167]
([-65.1459, -65.1274], [0.562958, 0.563167])


In [9]:
bspline2 = (-3 * u*u*u  + 3*u*u + 3*u + 1) / 6.0

 0.16666666666666666 + 0.5 t + 0.5 t² - 0.5 t³ + 𝒪(t⁴)

In [10]:
println(branchandbound(bspline2,dom,ϵ))
println(_minmax(bspline2, dom))

[0.0739386, 53.6042]
([0.0739212, 0.0740741], [53.5881, 53.6042])


In [11]:

bspline3 = -u*u*u / 6.0

 - 0.16666666666666666 t³ + 𝒪(t⁴)

In [12]:
println(branchandbound(bspline2,dom,ϵ))
println(_minmax(bspline2, dom))

[0.0739386, 53.6042]
([0.0739212, 0.0740741], [53.5881, 53.6042])
